In [1]:
import utils, data, simulation
import os
import os.path as op
import pandas as pd
import numpy as  np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
checkAxon = True
drawAllDist = False

In [3]:
#folder containing scripts
scriptpath = os.environ['SCRIPT_DATA']
outpath = op.join(scriptpath, 'data')
filename = 'stimulation_thresholds.csv'
df = pd.read_csv(op.join(outpath,'experiment_notes',filename))
df.columns = df.columns.str.strip()



In [4]:
# Remove the ones not used in the double electrode experiment

#calculate false positives for each patient
#catch trials
df = df[df['Selected']!=0]

#change column name
df= df.rename(columns= {'Patient No':'Subject'})
# Average multiple measurements for a given session
avg_data = (df
.groupby(['Subject','Session', 'Electrode'])
.mean())

In [5]:
# individual electrodes
avg_data.reset_index()

,Subject,Session,Electrode,Threshold,Selected
0,12-005,2019/05/29,A4,194.0,1.0
1,12-005,2019/05/29,A8,153.0,1.0
2,12-005,2019/05/29,D1,274.0,1.0
3,12-005,2019/05/29,E10,484.0,1.0
4,12-005,2019/05/29,F2,274.0,1.0
5,12-005,2019/10/9,A2,355.0,1.0
6,12-005,2019/10/9,B6,323.0,1.0
7,12-005,2019/10/9,F2,226.0,1.0
8,12-005,2019/10/9,F7,274.0,1.0
9,12-104,2019/05/30,B6,581.0,1.0


In [6]:
medians = (avg_data
.groupby(['Subject'])
.median())
medians.index

Index(['12-005', '12-104', '13-101'], dtype='object', name='Subject')

In [7]:
t_table = pd.DataFrame()
for subject in medians.index:
    query_string = 'Subject==' + '"' + subject + '"'
    print(query_string)  
    # select a single subject
    selected = avg_data.query(query_string)

    #calculate interquartile range of values in the 'points' column
    median = selected['Threshold'].median()
    q75, q25 = np.percentile(selected['Threshold'], [75 ,25])
    iqr = q75 - q25
    t_table = t_table.append({'Subject': subject, 'Median': median, 'Q25-Q75': (q25, q75),
                              'IQR': iqr, 'Minimum': np.min(selected['Threshold']), 
                             'Maximum': np.max(selected['Threshold'])}, ignore_index=True)

Subject=="12-005"
Subject=="12-104"
Subject=="13-101"


In [8]:
t_table

,IQR,Maximum,Median,Minimum,Q25-Q75,Subject
0,97.0,484.0,274.0,153.0,"(226.0, 323.0)",12-005
1,241.5,645.0,475.5,217.0,"(371.0, 612.5)",12-104
2,93.0,323.0,210.0,89.0,"(177.0, 270.0)",13-101


## Catch statistics

In [10]:
filename = 'S_12-005_12-104_13-101__collated_both_sessions.csv'

(double_x2, catch, subjectdata) = data.get_two_point(outpath, scriptpath, filename)

catch_means = pd.DataFrame()
for subject in medians.index:
    query_string = 'subject_id==' + '"' + subject + '"'
    print(query_string)  
    #select subject
    split = catch.query(query_string)
    # total number of trials per participant
    n= len(split)
    # times they perceived nothing when no stim
    hit = len(split[split['pts_number_processed']==0])
    # times they perceived >0 shapes
    fa = len(split[split['pts_number_processed']>0])
    print(n)
    print(hit)
    print(fa)
    catch_means = catch_means.append( {'subject_id': subject, 'false alarm rate': fa,
                             'total no of trials' : n, 'fa/n' : fa/n }, 
                             ignore_index= True)


subject_id=="12-005"
15
15
0
subject_id=="12-104"
12
10
2
subject_id=="13-101"
16
16
0


In [11]:
catch_means

,fa/n,false alarm rate,subject_id,total no of trials
0,0.000000,0.0,12-005,15.0
1,0.166667,2.0,12-104,12.0
2,0.000000,0.0,13-101,16.0
